# **Forelesning 6 - Long Short Term Memory modeller**
I denne forelesningen skal vi se nærmere på bruk av LSTM-modeller, og hvordan det kan brukes i f.eks. prediksjon av aksjekursen.

## Machine learning basics for tidsserier

* Husk alltid å starte med å **se** på data'en din. Bruker vi pandas er f.eks. denne metoden bra, da det viser de første $5$ radene og alle kolonnene for din dataframe.
```
df.head()
```
Alternativt kan man bruke *numpy*
```
array.shape # for å se formen til array'en din
array[:3]   # vise de tre første radene.
```

### Visualisering
Vi er alltid tjent med å vite hvordan tidsserie-data'en vår ser ut - og den beste måten da er ofte å visualisere hva som skjer. F.eks.
```
# Bruke matplotlib
fig, ax = plt.subplots()
ax.plot('insert det du skal plotte')

# Bruke pandas
fix, ax = plt.subplots()
df.plot('insert det du skal plotte', ax = ax)
```

Ved å visualisere data'en vår kan vi finne ut:
* Ser data'en vår rimelig ut?
* Mangler jeg noe data?
... og dette er viktig spørsmål å ha svar på før man i det hele tatt kan starte med analyse.

## Teori om LSTM
Se kompendium.

# Kode eksempel
Her skal vi se på hvordan vi kan predikere NVIDIA aksjekursen, selskapet som er delvis *medskyldig* i at AI har hatt den utviklingen den har hatt.

Så vi henter data fra [Yahoo Finance](https://finance.yahoo.com/quote/NVDA/history/), og henter mellom følgende datoer:

**Oppdater dato'er nærmere forelesningen**

# **1. Laste inn data**
Her bruker vi `pandas` til å lese inn en CSV-fil med aksjehistorikk fra **Yahoo Finance**.

`df.head()` viser de første 5 radene i datasettet, som viser aksjeprisen for en gitt dag (åpne-pris, høyeste, laveste, lukke-pris, justert sluttkurs og volum).

In [ ]:
# Starter med å importere to viktig libraries for LSTM
import tensorflow as tf
import pandas as pd

# Leser NVIDIA csv filen, henta fra: https://finance.yahoo.com/quote/NVDA/history/
df = pd.read_csv('NVDA.csv')

df.head()

In [ ]:
# Så kan vi se på shape til dataframe'n
df.shape

# **2. Velge relevante kolonner**
Nå trenger vi bare 'Date' og 'Close' kolonnene, siden vi kun ser på prediksjon av prisen når aksjehandelen lukkes for dagen.
Da gjør vi det på følgende måte

In [ ]:
df = df[['Date', 'Close']]

df

# **3. Datatyper i datasettet**
Noe vi har sett en del ganger i løpet av kurset nå, er at det er viktig med hvilken *type* data vi operer med. Det er fordi at nettverkene vi lager på *feedes* med data de kan forstå.

'Date' kolonner består av datoer, og er noe som kalles **ISO 8601 standard** (ÅÅÅÅ-MM-DD). Men selve data-typen er 'string' som vi ser under:

In [ ]:
# Vi sjekker hvilken data-type første element i 'Date' kolonnen er.
type(df['Date'][0])

## La oss gjør om 'Date' kolonnen til datetime.

In [ ]:
import pandas as pd

# Gjør om 'Date' kolonne til datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Date']

___
## Alternativ måte å gjøre om 'Date' til datetime

In [ ]:
# Date-kolonnen, er av typen 'object' eller 'str', som vi ser over. Så vi må endre dette til 'datetime'.

import datetime

### Funksjon som endrer et 'Object' med splitten '-' til datetime-variabel.
def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2]) # Splitter på hver int (heltall)
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
type(datetime_object)

In [ ]:
### Anvender funksjonen vår str_to_datetime på 'Date' kolonna, altså vi endrer det fra 'Object' til 'datetime64[ns]'

df['Date'] = df['Date'].apply(str_to_datetime)
df['Date']

___

## **4. Annen prosessering av data**
Her trenger vi ikke indeksen i vår `df`, så vi fjerner den.

In [ ]:
# Kvitter oss med indeksen - fordi vi ikke trenger det her
df.index = df.pop('Date')
df

# **5. Visualisering av data**

Nå som vi har lagd datasettet slik vi vil ha det, kan vi visulalisere hva som faktisk foregår.

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Close'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

# Vis plot
plt.show()

## Hvordan jobbe med tidsserie data i LSTM
Et konsept i LSTM, er at vi har lyst til å kjenne til noen tidligere verdier. Derfor lager vi en dataframe som inneholder verdier for idag ($Y$), igår ($Y-1$), i forigår ($Y-2$) og dagen før det ($Y-3$). Dette kaller vi ofte for *windowed datasets*.

In [ ]:
import matplotlib.pyplot as plt

# Definer tidssteg
time_steps = ["t-3", "t-2", "t-1", "t"]
values = ["Y_{t-3}", "Y_{t-2}", "Y_{t-1}", "Y_t"]

# Plassering av boksene
x_positions = range(len(time_steps))
y_position = 0

fig, ax = plt.subplots(figsize=(8, 2))

# Tegn boksene
for x, label in zip(x_positions, values):
    color = "blue" if "Y" in label else "green"
    ax.text(x, y_position, f"${label}$", ha="center", va="center",
            bbox=dict(boxstyle="round,pad=0.3", edgecolor=color, facecolor="white"))

# Tegn tidslinjen
ax.plot(x_positions, [y_position] * len(x_positions), "k-", linewidth=1)

# Tegn piler mellom boksene
for i in range(len(x_positions) - 1):
    ax.arrow(x_positions[i] + 0.2, y_position, 0.6, 0,
             head_width=0.1, head_length=0.1, fc="black", ec="black")

# Fjern akser
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

plt.title("Illustrasjon av rullerende vindu")
plt.show()

# **6. Lage et `windowed dataset`**

- Nå skal vi lage en funksjon som **omgjør tidsserie-data til et format som kan brukes i LSTM**, ved å lage et *vindu* på $n=3$ dager.

In [ ]:
import numpy as np

### Denne koden konverterer en time-series dataframe (df) av aksje-lukkepriser til en ny ###
### df hvor hver rad inneholder "et vindu" av tidligere verdier (n = 3), altså dag i-1, i-2 ###
### og i-3, i tillegg inneholder det target-value (dag i). Kilde: Gregg Hogg (YouTube) ###


def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while not last_time:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['Close'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df,
                                '2020-09-01',
                                '2024-05-28',
                                n=3)
windowed_df

# **7. Konvertere vårt `windowed dataset` til treningsdata**

- Nå skal vi konvertere vår `windowed_df` til NumPy arrays som kan brukes i LSTM.
- Splittes i **datoer** (`dates`), **features** (`X`) og **targets** (`Y`).

In [ ]:
# Her konverteres winowed_df om til våre features (X) og target (y). Kilde: Gregg Hogg (YouTube)

def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)

dates.shape, X.shape, y.shape

# **8. Lag trenings-, validerings- og testdata**

In [ ]:
q_80 = int(len(dates) * .80)
q_90 = int(len(dates) * .90)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

plt.legend(['Train', 'Validation', 'Test'])

# Vis plot
plt.show()

# **9. Lage og trene vår LSTM-modell**

- `Sequential`-modellen bygges steg for steg.
- **Input-lag** tar inn `3` verdier per datapunkt.
- **LSTM-lager** har `64` celler (recurrent for hukommelse).
- **Tre dense-lag** (`32` noder, ReLU som aktivering): lærer mønstrene i dataene.
- **Output-laget** (`Dense(1)`): Gir én enkelt prediksjon (altså, sluttkursen).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500)

# **10. Plotter de ulike prediksjonene mot faktiske data**

In [ ]:
train_predictions = model.predict(X_train).flatten()

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.legend(['Training Predictions', 'Training Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)


# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

In [ ]:
test_predictions = model.predict(X_test).flatten()

plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

# Vis plot
plt.show()

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Dollars $')
plt.title('NVIDIA Stock Closing Prices Over Time')

In [ ]:
y_test, test_predictions

___
# **Et annet eksempel fra Dehli**
La oss se på et eksempel-datasett som inneholder daglig værinformasjon fra Dehli.

Her skal jeg prøve å lage en LSTM modell av regndata fra klimadata fra Delhi fra 2013 til 2017.

In [ ]:
import tensorflow as tf
import pandas as pd

# Leser filene, hentet fra: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data/data
df_train = pd.read_csv('DailyDelhiClimateTrain.csv')
df_test = pd.read_csv('DailyDelhiClimateTest.csv')

# Konverter 'date' til datetime-format
df_train['date'] = pd.to_datetime(df_train['date'])
df_test['date'] = pd.to_datetime(df_test['date'])

# Etter mye pain fant jeg min bug:)
df_test = df_test[df_test['date'] != '2017-01-01']

# Slå sammen datasettene
df_dehli = pd.concat([df_train, df_test], axis=0)

# Sorter etter dato for å sikre riktig tidsrekkefølge
df_dehli = df_dehli.sort_values(by='date').reset_index(drop=True)

# Sjekk resultatet
print(df_dehli.head())
print(df_dehli.tail())

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_dehli.shape

## Hvilke datatyper har vi?

In [ ]:
df_dehli.info()

In [ ]:
import matplotlib.pyplot as plt
# Plotting
plt.figure(figsize=(10, 6))

# Plotter hver variabel
plt.plot(df_dehli['date'], df_dehli['meantemp'], label='Mean Temp', marker='o')
plt.plot(df_dehli['date'], df_dehli['humidity'], label='Humidity', marker='o')
plt.plot(df_dehli['date'], df_dehli['wind_speed'], label='Wind Speed', marker='o')
#plt.plot(df_train['date'], df_train['meanpressure'], label='Mean Pressure', marker='o')

# Set labels and title
plt.xlabel('Date')
plt.ylabel('Values')
plt.title('Weather Data Over Time')

# Rotate date labels for better readability
plt.xticks(rotation=45)

# Show legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
df_dehli

In [ ]:
df_dehli = df_dehli[['date', 'meantemp']]

In [ ]:
df_dehli

In [ ]:
# Kvitter oss med indeksen - fordi vi ikke trenger det her
df_dehli.index = df_dehli.pop('date')
df_dehli

In [ ]:
import numpy as np

### Denne koden konverterer en time-series dataframe (df) av aksje-lukkepriser til en ny ###
### df hvor hver rad inneholder "et vindu" av tidligere verdier (n = 3), altså dag i-1, i-2 ###
### og i-3, i tillegg inneholder det target-value (dag i). ###


def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date

  dates = []
  X, Y = [], []

  last_time = False
  while not last_time:
    df_subset = dataframe.loc[:target_date].tail(n+1)

    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['meantemp'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))

    if last_time:
      break

    target_date = next_date

    if target_date == last_date:
      last_time = True

  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates

  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]

  ret_df['Target'] = Y

  return ret_df

# Her må man begynne litt inn i datasettet mtp vindustørrelser
windowed_df = df_to_windowed_df(df_dehli,
                                '2013-01-04',
                                '2017-04-24',
                                n=3)
windowed_df

In [ ]:
# Plotting
plt.figure(figsize=(12, 6))

# Plot kolonnene 'Target-3', 'Target-2', 'Target-1', og 'Target'
plt.plot(windowed_df['Target Date'], windowed_df['Target-3'], label='Target-3', marker='o')
plt.plot(windowed_df['Target Date'], windowed_df['Target-2'], label='Target-2', marker='o')
plt.plot(windowed_df['Target Date'], windowed_df['Target-1'], label='Target-1', marker='o')
plt.plot(windowed_df['Target Date'], windowed_df['Target'], label='Target', marker='o')

# Setter på labels og titler
plt.xlabel('Target Date')
plt.ylabel('Values')
plt.title('Target Values Over Time')

plt.xticks(rotation=45)

plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
def windowed_df_to_date_X_y(windowed_dataframe):
  df_as_np = windowed_dataframe.to_numpy()

  dates = df_as_np[:, 0]

  middle_matrix = df_as_np[:, 1:-1]
  X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))

  Y = df_as_np[:, -1]

  return dates, X.astype(np.float32), Y.astype(np.float32)

dates, X, y = windowed_df_to_date_X_y(windowed_df)

dates.shape, X.shape, y.shape

In [ ]:
q_80 = int(len(dates) * .8)
q_90 = int(len(dates) * .9)

dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]

dates_val, X_val, y_val = dates[q_80:q_90], X[q_80:q_90], y[q_80:q_90]
dates_test, X_test, y_test = dates[q_90:], X[q_90:], y[q_90:]

plt.plot(dates_train, y_train)
plt.plot(dates_val, y_val)
plt.plot(dates_test, y_test)

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Mean temp.')
plt.title('Mean temperature over time')

plt.legend(['Train', 'Validation', 'Test'])

# Vis plot
plt.show()

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
len(X_train)

In [ ]:
y_train[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# Definerer Early Stopping kriterier
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model = Sequential([layers.Input((3, 1)),
                    layers.LSTM(64),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(32, activation='relu'),
                    layers.Dense(1)])

model.compile(loss='mse',
              optimizer='adam',
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=500, callbacks=[early_stopping])

In [ ]:
train_predictions = model.predict(X_train).flatten()

plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.legend(['Training Predictions', 'Training Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.title('Dehli temperature prediction over time')

In [ ]:
val_predictions = model.predict(X_val).flatten()

plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.legend(['Validation Predictions', 'Validation Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)


# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.title('Dehli temperature prediction over time')

In [ ]:
test_predictions = model.predict(X_test).flatten()

plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Testing Predictions', 'Testing Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.title('Dehli temperature prediction over time')

# Vis plot
plt.show()

In [ ]:
plt.plot(dates_train, train_predictions)
plt.plot(dates_train, y_train)
plt.plot(dates_val, val_predictions)
plt.plot(dates_val, y_val)
plt.plot(dates_test, test_predictions)
plt.plot(dates_test, y_test)
plt.legend(['Training Predictions',
            'Training Observations',
            'Validation Predictions',
            'Validation Observations',
            'Testing Predictions',
            'Testing Observations'])

# Formaterer x-aksen til å vise datoer og roterer dem
plt.gcf().autofmt_xdate()  # Automatisk rotasjon av datoer fordi det ser pent ut:)

# Adder labels og title
plt.xlabel('Date')
plt.ylabel('Mean temperature')
plt.title('Dehli temperature prediction over time')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, test_predictions)

# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, test_predictions)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

# R-squared (R²)
r2 = r2_score(y_test, test_predictions)

# Outputs
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-squared (R²): {r2:.4f}")